In [0]:
path ="/Volumes/workspace/etl_practice/my_file/books-csv"

In [0]:
df = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("/Volumes/workspace/etl_practice/my_file/books-csv/export_001.csv")
display(df)


In [0]:
df.count()

In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA etl_practice;

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("books")

In [0]:
%sql
select *from books

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("workspace.etl_practice.books_2")

In [0]:
%sql
drop table workspace.etl_practice.books_2

In [0]:
df2 = spark.read.format("csv").option("header", "true").option("delimiter", ";").load(f"{path}/export_002.csv")
display(df2)
df2.write.format("delta").mode("append").saveAsTable("books")

In [0]:
%sql
select *from books

In [0]:
df3 = spark.read.format("csv").option("header", "true").option("delimiter", ";").load(f"{path}/export_003.csv")
display(df3)

In [0]:
df3.createOrReplaceTempView("book_3")

In [0]:
%sql
select *from book_3

In [0]:
%sql
insert into books select *from book_3

In [0]:
%sql
SELECT *FROM books

In [0]:
%sql
COPY INTO books
FROM '/Volumes/workspace/etl_practice/my_file/books-csv/export_004.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS (
  'header' = 'true',
  'delimiter' = ';'
);

In [0]:
%sql
SELECT *FROM books

In [0]:
df = (
  spark.readStream
       .format("cloudFiles")
       .option("cloudFiles.format", "csv")
       .option("cloudFiles.schemaLocation", f"/Volumes/workspace/etl_practice/my_file/_schemas")
       .option("header", "true")
       .option("delimiter", ";")
       .load(f"{path}/")
)

In [0]:
df.createOrReplaceTempView("test2")

In [0]:
df.writeStream \
  .format("delta") \
  .option("checkpointLocation", f"/Volumes/workspace/etl_practice/my_file/_schemas/test2") \
  .table("books3")

In [0]:
%sql
select *from test2

In [0]:
/Volumes/workspace/etl_practice/my_file/books-csv/

In [0]:
base_path = "/Volumes/workspace/etl_practice"

schema_path = f"{base_path}/my_file/_schemas/dev_table"
checkpoint_path = f"{base_path}/my_file/_checkpoints/dev_table"
source_path = f"{base_path}/my_file/books-csv"

(
  spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", schema_path)
    .load(source_path)
    .writeStream
    .option("checkpointLocation", checkpoint_path)
    .trigger(availableNow=True)
    .toTable("workspace.etl_practice.dev_table")
)

In [0]:
df3.printSchema()

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, StructField

base_path = "/Volumes/workspace/etl_practice"

schema_path = f"{base_path}/my_file/_schemas/dev_table_schema"
checkpoint_path = f"{base_path}/my_file/_checkpoints/dev_table"
source_path = f"{base_path}/my_file/books-csv"

json_schema = StructType([
    StructField("book_id", StringType(), True),
    StructField("title", StringType(), True),
    StructField("author", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", IntegerType(), True)
])

(
  spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", schema_path)
    .option("delimiter", ";")
    .schema(json_schema)
    .load(source_path)
    .writeStream
    .option("checkpointLocation", checkpoint_path)
    .trigger(availableNow=True)
    .toTable("workspace.etl_practice.dev_table")
)


In [0]:
%sql
select count(*) from workspace.etl_practice.dev_table

In [0]:
df3 = spark.read.format("csv").option("header", "true").option("delimiter", ";").load(f"/Volumes/workspace/etl_practice/my_file/books-csv/export_005.csv")
display(df3)

In [0]:
%run ../config/sqlconfig

In [0]:
%sql drop table dev_table

In [0]:
%sql
drop table books

In [0]:
%sql
drop table books3

In [0]:
%sql 
SELECT *FROM silver_fact_sales

In [0]:
%sql
select *,
  row_number() over(
    partition by sales_id, transaction_ts, date_key, product_key, customer_key, store_key, payment_key, quantity, unit_price, discount_pct, net_sales_amount, tax_amount
    order by transaction_ts
  ) as num_rows
from silver_fact_sales

In [0]:
%sql
select *,
  row_number() over(
    partition by sales_id, transaction_ts, date_key, product_key, customer_key, store_key, payment_key, quantity, unit_price, discount_pct, net_sales_amount, tax_amount
    order by transaction_ts
  ) as num_rows
from fact_sales

In [0]:
%sql
-- insert into silver_fact_sales values(2,	'01-01-2024 09:06',	20240101,	102,	1001,	10,	1,	2,	50.00,	0.00,	100.00,	10.00);
-- insert into silver_fact_sales values(4,	'01-01-2024 11:22',	20240101,	104,	1003,	12,	3,	3,	15.00,	0.00,	45.00,	4.50);
